# Configuracion de grafica a usar

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="1";

# Importacion de librerias

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider
import h5py
import numpy as np

%matplotlib inline
import sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from LTC import *
sys.path.append('../')
from Datasets_utils.DatasetsLoader import VideoDataGenerator

# Configuraciones para Tensorflow y Keras

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [5]:
tf.debugging.set_log_device_placement(False)

In [ ]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

# Carga de Datos

In [ ]:
root_path = "/home/jefelitman/DataSets/ut/videos_set_1"
root_path

In [ ]:
batch_size = 12
original_size = [171,128]
size = [112,112]
frames = 16
canales = 3

In [ ]:
def custom_steps_temporal(frames):
    mitad = len(frames)//2
    paso = len(frames)//16
    videos = []
    for i in range(paso):
        indices = range(i, len(frames),paso)
        videos.append([frames[j] for j in indices[:16]])
    return videos

In [ ]:
def custom_temp_crop_unique(frames):
    mitad = len(frames)//2
    paso = len(frames)//16
    indices = sorted(list(range(mitad, -1,-paso)) + list(range(mitad+paso, len(frames),paso)))
    indices = indices[len(indices)//2 - 8 : len(indices)//2 + 8]
    return [[frames[i] for i in indices]]

In [ ]:
def half_video_temporal(frames):
    mitad = len(frames)//2
    return [frames[mitad-8*2:mitad+8*2]]

In [ ]:
def video_transf(video):
    escalador = MinMaxScaler()
    new_video = video.reshape((video.shape[0]*video.shape[1]*video.shape[2]*video.shape[3],1))
    new_video = escalador.fit_transform(new_video)
    return new_video.reshape((video.shape[0],video.shape[1],video.shape[2],video.shape[3]))

In [ ]:
def flip_vertical(volume):
    return np.flip(volume, (0, 2))[::-1]

In [ ]:
def corner_frame_crop(original_width, original_height):
    x = original_width-112
    y = original_height-112
    return [[x//2, original_width - x//2 -1, y//2, original_height-y//2],
            [x//2, original_width - x//2 -1, y//2, original_height-y//2]
           ]

In [ ]:
dataset = VideoDataGenerator(directory_path = root_path, 
                             table_paths = None, 
                             batch_size = batch_size, 
                             original_frame_size = original_size, 
                             frame_size=size, 
                             video_frames = frames, 
                             temporal_crop = ("custom", custom_steps_temporal),
                             video_transformation = [("augmented",flip_vertical)],
                             frame_crop = ("custom", corner_frame_crop), 
                             shuffle = True, 
                             conserve_original = False)

# Red Neuronal LTC

### Construccion del modelo

In [ ]:
#Entrada de la red neuronal
video_shape = tuple([frames]+size[::-1]+[canales])
dropout = 0.5
lr = 1e-3
weigh_decay = 5e-3

ltc_save_path = '/home/jefelitman/Saved_Models/trained_ut/Encoder/Inception/inception_enhan/LTC-enhan-noMaxPoolT-SLTEnc_Seq_split1_{w}x{h}x{f}_softmax_sgd_'.format(
        w=size[0], h=size[1],f=frames)
if canales == 3:
    ltc_save_path += 'RGB_'
else:
    ltc_save_path += 'B&N_'

ltc_save_path += 'lr={l}__dec-ori_2center-frame-crop_video-flip_temporal-dynamic_batchNorm_pretrained-c3d'.format(l = lr)

#Creacion de la carpeta donde se salvara el modelo
if not os.path.isdir(ltc_save_path):
    os.mkdir(ltc_save_path)
model_saves_path = os.path.join(ltc_save_path,'model_saves')
if not os.path.isdir(model_saves_path):
    os.mkdir(model_saves_path)
ltc_save_path

In [6]:
lr = 1e-3
weigh_decay = 5e-3
#Parametros para la compilacion del modelo
optimizador = keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
#optimizador = keras.optimizers.Adam(learning_rate=lr)
perdida = keras.losses.SparseCategoricalCrossentropy()
precision = keras.metrics.SparseCategoricalAccuracy()

In [13]:
entrada = keras.Input(shape=(300,224,224,3),batch_size=1,
                     name="Input_video")
#Conv1
x = keras.layers.Conv3D(filters=16, kernel_size=3, padding="same", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          name='conv3d_1')(entrada)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2), name='max_pooling3d_1')(x)

#Conv2
x = keras.layers.Conv3D(filters=32, kernel_size=3, padding="same", activation="relu", 
                      kernel_regularizer=keras.regularizers.l2(weigh_decay),
                      name='conv3d_2')(x)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2), name='max_pooling3d_2')(x)

#Conv3
x = keras.layers.Conv3D(filters=64, kernel_size=3, padding="same", activation="relu", 
                      kernel_regularizer=keras.regularizers.l2(weigh_decay),
                      name='conv3d_3')(x)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2),name='max_pooling3d_3')(x)

#Conv4
x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="same", activation="relu", 
                      kernel_regularizer=keras.regularizers.l2(weigh_decay),
                      name='conv3d_4')(x)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2),name='max_pooling3d_4')(x)

ltc = keras.Model(entrada, x, name="LTC_original")

In [ ]:
ltc = get_LTC_encoder_slt_I(video_shape, len(dataset.to_class),dropout, weigh_decay, 256, 512, True)

In [8]:
#Compilacion del modelo
ltc.compile(optimizer = optimizador,
           loss = perdida,
           metrics = [precision])

In [9]:
#keras.utils.plot_model(ltc, 'LTC.png', show_shapes=True)

In [10]:
#ltc = keras.models.load_model('/home/jefelitman/Saved_Models/trained_ut/Inception/conv_channels/LTC-incept-channels_split1_112x112x16_softmax_sgd_RGB_lr=0.001_decreased-original_2center-frame-crop_video-flip_temporal-dynamic_batchNorm_pretrained-c3d/ltc_final_1.h5')

In [14]:
ltc.summary()

Model: "LTC_original"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_video (InputLayer)     [(1, 300, 224, 224, 3)]   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (1, 300, 224, 224, 16)    1312      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (1, 300, 112, 112, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (1, 300, 112, 112, 32)    13856     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (1, 300, 56, 56, 32)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (1, 300, 56, 56, 64)      55360     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (1, 300, 28, 28, 64)     

In [12]:
ltc.predict(np.zeros((1,300,224,224,3)))

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0.

### Cargo los pesos pre entrenados

##### Pesos del C3D

In [ ]:
c3d_weights = h5py.File('/home/jefelitman/Saved_Models/c3d-sports1M_weights.h5', 'r')
print(c3d_weights.keys())

In [ ]:
c3d_weights['layer_0'].keys()

In [ ]:
weights = []
for capa in ['layer_0','layer_2','layer_4','layer_5','layer_5']:
    weights.append([
        np.moveaxis(np.r_[c3d_weights[capa]['param_0']], (0,1),(4,3)), #Cambio los ejes porque c3d estan con canales primero
        np.r_[c3d_weights[capa]['param_1']]
                   ])
for index, capa in enumerate(['conv3d_1','conv3d_2','conv3d_3','conv3d_4','conv3d_5']):
    ltc.get_layer(capa).set_weights(weights[index])

##### Pesos de la InceptionV3

### Entrenamiento de la red con el generador

In [ ]:
#Funcion customizadas para el entrenamiento del modelo      
class custom_callback(keras.callbacks.Callback):
    def __init__(self):
        self.accuracies = []
        self.losses = []
        self.val_accuracies = []
        self.val_loss = []
        
    def on_batch_end(self, batch, logs):
        corte = dataset.train_batches//3 + 1
        if batch == corte or batch == corte*2:
            keras.backend.set_value(optimizador.lr, optimizador.lr.numpy()*0.1)
            for i in ['conv3d_1','conv3d_2','conv3d_3','conv3d_4','conv3d_5','dense_8','dense_9','dense_10']:
                weigh_decay = ltc.get_layer(i).kernel_regularizer.get_config()['l2'] * 0.1
                ltc.get_layer(i).kernel_regularizer = keras.regularizers.l2(weigh_decay)
            print("\n","Actual LR: ", str(optimizador.lr.numpy()))
            
        self.accuracies.append(logs['sparse_categorical_accuracy'])
        self.losses.append(logs['loss'])
        
    def on_epoch_begin(self, epoch, logs):
        keras.backend.set_value(optimizador.lr, 0.001)
            
    def on_epoch_end(self,batch, logs):
        self.val_accuracies.append(logs['val_sparse_categorical_accuracy'])
        self.val_loss.append(logs['val_loss'])

funciones = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_saves_path,'ltc_epoch_{epoch}.h5'),
        save_best_only=True,
        monitor='val_sparse_categorical_accuracy',
        verbose=1),
    keras.callbacks.CSVLogger(os.path.join(ltc_save_path,'output.csv')),
    custom_callback()
]

In [ ]:
epoch = 1
historial = ltc.fit(x = dataset.get_train_generator(canales),
                 steps_per_epoch=dataset.train_batches,
                 epochs=epoch,
                 callbacks=funciones,
                 validation_data= dataset.get_test_generator(canales),
                 validation_steps=dataset.test_batches,
                 max_queue_size=batch_size%24)

### Guardado del modelo 

In [ ]:
#Salvado final definitivo del modelo una vez se detenga
ltc.save(os.path.join(ltc_save_path,"ltc_final_{e}.h5".format(e=epoch)))

### Graficas de los resultados de entrenamiento

In [ ]:
fig = plt.figure()
plt.plot(funciones[-1].losses,'k--')
rango = [i*dataset.train_batches-1 for i in range(1,epoch+1)]
plt.plot(rango, funciones[-1].val_loss,'bo')
plt.title('Loss over steps')
plt.legend(labels=["Loss","Test Loss"])
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_loss_steps_{e}.png'.format(e=dataset.train_batches)))

In [ ]:
fig = plt.figure()
plt.plot(funciones[-1].accuracies,'k--')
rango = [i*dataset.train_batches-1 for i in range(1,epoch+1)]
plt.plot(rango, funciones[-1].val_accuracies,'bo')
plt.title('Accuracy over steps')
plt.legend(labels=["Accuracy","Test Accuracy"])
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_accuracy_steps_{e}.png'.format(e=dataset.train_batches)))

### Evaluacion del entrenamiento